In [36]:
import pandas as pd
import numpy as np

# a) Generate a Dataset

Randomly generate a dataset (dataframe) with eight columns and 50,000 rows. Each column should be a categorical variable (of arbitrary name) with three levels (of arbitrary names) in approximately equal proportions.

## i. Random Generation

In [37]:
examples=50000
features=8
num_categories=3

#Encode each of the three categories as integers
#Generate random floats from 0-1. Multiply by num_categories to get random numbers over the right range, then round down to get integers.
rand_arr = np.floor(np.random.rand(examples,features)*num_categories).astype(int)
rand_arr

array([[1, 1, 0, ..., 0, 1, 1],
       [2, 0, 2, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 2, 2],
       [0, 2, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 2, 1, 2]])

## ii. Convert to a Pandas Dataframe

In [38]:
df = pd.DataFrame(rand_arr)

#Rename the columns for clarity
col_numbers = range(0,8)
col_names = [f"Column {col}" for col in col_numbers]
#generate a dictionary of column numbers and names to pass to the dataframe rename method
columns_rename = {number:name for (number,name) in zip(col_numbers, col_names)}
df.rename(columns=columns_rename, inplace=True)

df


,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7
0,1,1,0,2,0,0,1,1
1,2,0,2,2,2,0,0,0
2,2,0,0,0,0,0,1,1
3,2,1,1,0,2,1,0,2
4,2,2,1,2,0,1,0,0
...,...,...,...,...,...,...,...,...
49995,2,2,0,2,2,1,1,0
49996,0,2,2,2,2,0,0,0
49997,1,1,1,2,1,1,2,2
49998,0,2,0,2,0,1,1,1


# b) Verify Distribution

Veryify that the proportions of each value are similar for each of the eight columns.

In [39]:
df_counts = pd.DataFrame()

#for each of the integer encoded labels, create a series with the number of instances of that label for each column and append that to a new dataframe
for val in range(0,num_categories):
    df_counts = df_counts.append(df[df == val].count(), ignore_index=True)

df_counts.index.set_names('Category Label', inplace=True)
print("Count of Category Labels by Column:")
df_counts

Count of Category Labels by Column:


,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7
Category Label,,,,,,,,
0,16502.0,16611.0,16769.0,16758.0,16477.0,16691.0,16795.0,16632.0
1,16703.0,16659.0,16651.0,16538.0,16750.0,16763.0,16781.0,16775.0
2,16795.0,16730.0,16580.0,16704.0,16773.0,16546.0,16424.0,16593.0


The proportions of each label are are similar for each column: within expectations of what we would see from random generation.

# C) Unique Combinations

How many unique rows (i.e., permutations of category levels) are possible?

In [41]:
print("There are {} unique permutations.".format(num_categories**features))

There are 6561 unique permutations.
